In [27]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Fetching the dataset

In [28]:
housing_data = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(housing_data["data"], housing_data["target"])
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train)

scaler = StandardScaler()
X_train_zscore = scaler.fit_transform(X_train)
X_validation_zscore = scaler.transform(X_validation)
X_test_zscore = scaler.transform(X_test)

## Fining tuning a neural network hyperparameters

It is possible to use at least two search methods to tune hyperparameters:

> 1) Random serach cross-validation;
> 2) Grid search cross-validation.

To compare the neural network models selected by the methods stated above, we first estimate the weights of a feed-forward neural network with one hidden layer and 30 hidden units per layer. This process is done by making use of the sklearn package.

In [29]:
# build a wrapper for the sequential API
## FFNN with 1 hidden unit, and 30 hidden units (neurons) in each hidden layer
def ffnn_dynamic_hidden_layers(n_hidden=1,
                               n_neurons=30,
                               learning_rate=3e-3,
                               input_shape=[8],
                               loss="mse",
                               activation="relu"):
    
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation=activation)) 
        
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss=loss, optimizer=optimizer)
    
    return model

# transform the FNN wrapper into a sklearn regressor object
ffnn_dynamic_hidden_layers_reg = keras.wrappers.scikit_learn.KerasRegressor(ffnn_dynamic_hidden_layers)

# fit the default model "fnn_dynamic_hidden_layers"
ffnn_dynamic_hidden_layers_reg.fit(X_train_zscore,
                                   y_train,
                                   epochs=100,
                                   validation_data=(X_validation_zscore, y_validation),
                                   callbacks=[keras.callbacks.EarlyStopping(patience=10)]) 
mse_test = ffnn_dynamic_hidden_layers_reg.score(X_test_zscore,
                                                y_test) 

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 1s 62us/sample - loss: 1.1385 - val_loss: 0.6928
Epoch 2/100
11610/11610 [==============================] - 0s 29us/sample - loss: 0.6183 - val_loss: 0.6216
Epoch 3/100
11610/11610 [==============================] - 0s 29us/sample - loss: 0.5723 - val_loss: 0.5749
Epoch 4/100
11610/11610 [==============================] - 0s 29us/sample - loss: 0.5151 - val_loss: 0.5386
Epoch 5/100
11610/11610 [==============================] - 0s 29us/sample - loss: 0.4891 - val_loss: 0.5165
Epoch 6/100
11610/11610 [==============================] - 0s 29us/sample - loss: 0.4686 - val_loss: 0.4993
Epoch 7/100
11610/11610 [==============================] - 0s 29us/sample - loss: 0.4527 - val_loss: 0.4834
Epoch 8/100
11610/11610 [==============================] - 0s 29us/sample - loss: 0.4459 - val_loss: 0.4753
Epoch 9/100
11610/11610 [==============================] - 0s 29us/sample - loss: 0.433

Once we have estimated the parameters of the benchmark network, we can use our first approach for hyperparameter search.

The hyperparameters of this specific network are:

> 1) n_hidden: Number of hidden units;
> 2) n_neurons: Number of hidden units per hidden neuron;
> 3) learning_rate: rate at which the steps of the gradient are taken;
> 4) loss: Loss function to be used on the optimization process;
> 5) activation: activation function to be used between hidden layers

For the purpose of this empirical exercise we will use the search methods on hyperparamenters 1), 2) and 3). 

Our first method will be the random serch cross-validation. Once we have build the regressor object on the sklearn sintax, we can make use of the "RandomSearchCV" function to search over the prespecified parameters "param_distribs". Ohter two important parameters of the "RandomSearchCV" function are: "n_iter" and "cv". The "n_iter" parameter controls the number of random samples to pick from the hyperparameter set, and the "cv" parameter controls the number of folds to be used on the cross-validations setup.

In [ ]:
param_distribs = {"n_hidden": [0, 1, 2, 3],
                  "n_neurons": np.arange(1, 100),
                  "learning_rate": reciprocal(3e-4, 3e-2)}

rnd_search_cv = RandomizedSearchCV(ffnn_dynamic_hidden_layers_reg,
                                   param_distribs,
                                   n_iter=10,
                                   cv=3) 
rnd_search_cv.fit(X_train_zscore,
                  y_train,
                  epochs=100,
                  validation_data=(X_validation_zscore, y_validation),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 0s 62us/sample - loss: 5.5735 - val_loss: 4.0403
Epoch 2/100
7740/7740 [==============================] - 0s 29us/sample - loss: 3.3062 - val_loss: 2.5388
Epoch 3/100
7740/7740 [==============================] - 0s 29us/sample - loss: 2.1389 - val_loss: 1.7249
Epoch 4/100
7740/7740 [==============================] - 0s 29us/sample - loss: 1.5037 - val_loss: 1.2661
Epoch 5/100
7740/7740 [==============================] - 0s 29us/sample - loss: 1.1438 - val_loss: 0.9996
Epoch 6/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.9335 - val_loss: 0.8409
Epoch 7/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.8068 - val_loss: 0.7451
Epoch 8/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.7291 - val_loss: 0.6865
Epoch 9/100
7740/7740 [==============================] - 0s 29us/sample - loss: 0.6804 - val_loss: 0.650

In [1]:
rnd_search_cv.best_params_
rnd_search_cv.best_score_
model = rnd_search_cv.best_estimator_.model

NameError: name 'rnd_search_cv' is not defined